## Short and Simple Subqueries
- learn about subqueries in the SELECT, FROM, and WHERE clauses. 
- gain an understanding of when subqueries are necessary to construct dataset and where to best include them in queries.

In [3]:
import pandas as pd
import sqlite3
%reload_ext sql
%sql sqlite:///database.sqlite

con=sqlite3.connect("database.sqlite")
mycur = con.cursor()
mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
available_table=(mycur.fetchall())
con.close() 
available_table


[('Country',),
 ('League',),
 ('Match',),
 ('Player',),
 ('Player_Attributes',),
 ('Team',),
 ('Team_Attributes',),
 ('sqlite_sequence',)]

### Filtering using scalar subqueries

In [7]:
%%sql
SELECT 
    -- Select the date, home goals, and away goals scored
    date,
    home_goal,
    away_goal
FROM  Match
-- Filter for matches where total goals exceeds 3x the average
WHERE (home_goal + away_goal) > 
       (SELECT 3 * AVG(home_goal + away_goal)
        FROM Match)
LIMIT 10; 

 * sqlite:///database.sqlite
Done.


date,home_goal,away_goal
2011-10-29 00:00:00,4,5
2009-11-22 00:00:00,9,1
2010-01-16 00:00:00,7,2
2011-08-28 00:00:00,8,2
2012-12-29 00:00:00,7,3
2013-05-19 00:00:00,5,5
2013-12-14 00:00:00,6,3
2014-03-22 00:00:00,3,6
2014-08-30 00:00:00,3,6
2016-01-23 00:00:00,4,5


### Filtering using a subquery with a list

In [18]:
%%sql
SELECT * FROM Team LIMIT 1;


 * sqlite:///database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN


In [29]:
%%sql
SELECT DISTINCT hometeam_id FROM Match
LIMIT 5

 * sqlite:///database.sqlite
Done.


hometeam_id
9987
10000
9984
9991
7947


In [47]:
%%sql
SELECT 
    team_long_name,
    team_short_name
FROM Team 
WHERE team_api_id != --NOT IN
        (SELECT DISTINCT hometeam_id 
         FROM Match)
LIMIT 10;


 * sqlite:///database.sqlite
Done.


team_long_name,team_short_name
Beerschot AC,BAC
SV Zulte-Waregem,ZUL
Sporting Lokeren,LOK
KSV Cercle Brugge,CEB
RSC Anderlecht,AND
KAA Gent,GEN
RAEC Mons,MON
FCV Dender EH,DEN
Standard de Liège,STL
KV Mechelen,MEC


In [50]:
%%sql
SELECT
    -- Select the team long and short names
    team_long_name,
    team_short_name
FROM team
-- Filter for teams with 8 or more home goals
WHERE team_api_id IN
       (SELECT hometeam_ID 
       FROM match
       WHERE home_goal >= 8);

 * sqlite:///database.sqlite
Done.


team_long_name,team_short_name
Chelsea,CHE
Southampton,SOU
Tottenham Hotspur,TOT
Real Madrid CF,REA
FC Barcelona,BAR
PSV,PSV
SL Benfica,BEN
FC Bayern Munich,BMU
Celtic,CEL
Manchester United,MUN


### Joining Subqueries in FROM

In [53]:
%%sql
SELECT
-- Select country name and the count match IDs
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER JOIN (SELECT country_id, id
           FROM match
           -- Filter the subquery by matches with 10+ goals
           WHERE (home_goal + away_goal) >=10) AS sub
ON c.id = sub.country_id
GROUP BY country_name;

 * sqlite:///database.sqlite
Done.


country_name,matches
England,4
France,1
Germany,1
Netherlands,2
Scotland,1
Spain,5


In [58]:
%%sql
SELECT
    country,
    date,
    home_goal,
    away_goal
FROM 
    (SELECT c.name AS country, 
                m.date, 
                m.home_goal, 
                m.away_goal,
              (m.home_goal + m.away_goal) AS total_goals
       FROM match AS m
       LEFT JOIN country AS c
       ON m.country_id = c.id) AS subq

WHERE total_goals >= 10;

 * sqlite:///database.sqlite
Done.


country,date,home_goal,away_goal
England,2009-11-22 00:00:00,9,1
England,2011-08-28 00:00:00,8,2
England,2012-12-29 00:00:00,7,3
England,2013-05-19 00:00:00,5,5
France,2009-11-08 00:00:00,5,5
Germany,2013-03-30 00:00:00,9,2
Netherlands,2010-10-24 00:00:00,10,0
Netherlands,2011-11-06 00:00:00,6,4
Scotland,2010-05-05 00:00:00,6,6
Spain,2013-10-30 00:00:00,7,3


### subquery to the SELECT clause

In [59]:
%%sql

SELECT 
	l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND(AVG(home_goal + away_goal), 2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY l.name;

 * sqlite:///database.sqlite
Done.


league,avg_goals,overall_avg
Belgium Jupiler League,2.5,2.77
England Premier League,2.77,2.77
France Ligue 1,2.46,2.77
Germany 1. Bundesliga,3.16,2.77
Italy Serie A,2.72,2.77
Netherlands Eredivisie,3.2,2.77
Poland Ekstraklasa,2.64,2.77
Portugal Liga ZON Sagres,2.37,2.77
Scotland Premier League,2.75,2.77
Spain LIGA BBVA,2.75,2.77


In [60]:
%%sql
SELECT 
    l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND(AVG(home_goal + away_goal), 2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY l.name;SELECT 
    l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND(AVG(home_goal + away_goal), 2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY l.name;

 * sqlite:///database.sqlite
Done.
Done.


league,avg_goals,overall_avg
Belgium Jupiler League,2.5,2.77
England Premier League,2.77,2.77
France Ligue 1,2.46,2.77
Germany 1. Bundesliga,3.16,2.77
Italy Serie A,2.72,2.77
Netherlands Eredivisie,3.2,2.77
Poland Ekstraklasa,2.64,2.77
Portugal Liga ZON Sagres,2.37,2.77
Scotland Premier League,2.75,2.77
Spain LIGA BBVA,2.75,2.77


In [62]:
%%sql
SELECT 
	-- Select the stage and average goals for each stage
	m.stage,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select the average overall goals for the 2012/2013 season
    ROUND((SELECT AVG(home_goal + away_goal) 
           FROM match 
           WHERE season = '2012/2013'),2) AS overall
FROM match AS m
-- Filter for the 2012/2013 season
WHERE m.season = '2012/2013'
-- Group by stage
GROUP BY m.stage
LIMIT 10;

 * sqlite:///database.sqlite
Done.


stage,avg_goals,overall
1,2.68,2.77
2,2.65,2.77
3,2.83,2.77
4,2.8,2.77
5,2.61,2.77
6,2.78,2.77
7,2.69,2.77
8,3.09,2.77
9,2.7,2.77
10,2.96,2.77


In [65]:
%%sql
SELECT 
    -- Select the stage and average goals from the subquery
    stage,
    ROUND(avg_goals,2) AS avg_goals
FROM 
    -- Select the stage and average goals in 2012/2013
    (SELECT
         stage,
         AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE 
    -- Filter the main query using the subquery
    s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                    FROM match WHERE season = '2012/2013');

 * sqlite:///database.sqlite
Done.


stage,avg_goals
3,2.83
4,2.8
6,2.78
8,3.09
10,2.96
11,2.92
12,3.23
17,2.85
20,2.96
21,2.9


In [67]:
%%sql
SELECT 
    -- Select the stage and average goals from s
    s.stage,
    ROUND(s.avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
    -- Select the stage and average goals in 2012/2013 from match
    (SELECT
         stage,
         AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE 
    -- Filter the main query using the subquery
    s.avg_goals >  (SELECT AVG(home_goal + away_goal) 
                    FROM match WHERE season = '2012/2013');

 * sqlite:///database.sqlite
Done.


stage,avg_goal,overall_avg
3,2.83,2.7726993865030676
4,2.8,2.7726993865030676
6,2.78,2.7726993865030676
8,3.09,2.7726993865030676
10,2.96,2.7726993865030676
11,2.92,2.7726993865030676
12,3.23,2.7726993865030676
17,2.85,2.7726993865030676
20,2.96,2.7726993865030676
21,2.9,2.7726993865030676
